## Final Project - Audio Visualization

### By Rinki and Zohaib 

#### Project Introduction
* Converting values of an audio's spectrogram into decibels
* Creating a visualization of these decibel values for the duration of the audio's time

### TODO
  
  - get familiar with audio visualization apis and libraries

### Reading in Audio

### Applying STFT Function